In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 40.3 MB/s eta 0:00:00


In [ ]:

import pennylane as qml
import pennylane.numpy as np
import matplotlib.pyplot as plt

In [122]:
def hydrogen_hamiltonian(coordinates):
    """Calculates the qubit Hamiltonian of the hydrogen molecule.

    Args:
        coordinates (list(float)): Cartesian coordinates of each hydrogen molecule.
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (qml.Hamiltonian): A PennyLane Hamiltonian.
    """
    return qml.qchem.molecular_hamiltonian(["H", "H","H"], coordinates, charge=1, basis="STO-3G")[0]


def hf(electrons, num_qubits):
    """Calculates the Hartree-Fock state of the hydrogen molecule.

    Args:
        electrons (int): The number of electrons in the hydrogen molecule.
        num_qubits (int): The number of qubits needed to represent the hydrogen molecule Hamiltonian.

    Returns:
        (numpy.tensor): The HF state.
    """
    # Put your solution here #
    return qml.qchem.hf_state(electrons=electrons, orbitals=num_qubits)

In [123]:
def num_electrons(charge):
    """The total number of electrons in the hydrogen molecule.

    Args:
        charge (int): The electric charge given to the hydrogen molecule.

    Returns:
        (int): The number of electrons.
    """
    return 4-charge

In [124]:
def depth(qnode):
    def _fn(*args, **kwargs):
        qnode.construct(args, kwargs)
        return qnode.qtape.get_depth()
    return _fn

Using exact diagonalization to get the ground state and first excited state.

In [127]:
L=1.5
coordinates = np.array([-L/2/np.sqrt(3), L/2, 0.0, L/np.sqrt(3), 0.0, 0.0,-L/2/np.sqrt(3), -L/2, 0.0])
charge=0
hamiltonian = hydrogen_hamiltonian(np.array(coordinates))
H_matrix=qml.matrix(hamiltonian)

vals, vecs = np.linalg.eigh(H_matrix)

inds=np.argsort(vals)
eng=vals[inds]
vec=vecs[:,inds]

print('The ground state energy:',eng[0])


The ground state energy: -1.232574516702284


In [129]:
sz=qml.qchem.spinz(len(hamiltonian.wires))
SZ=qml.matrix(sz)
for i in range(len(eng)): # Finding first excited state with -1 Sz
    Sz=vec[:,i].dot(SZ.dot(np.transpose(np.conjugate(vec[:,i]))))
    if np.abs(Sz+1) < 1e-5:
        break
print('The first excied state energy:',eng[i])

The first excied state energy: -0.5197267013429419


In [148]:
hf_state = hf(2, len(hamiltonian.wires))

In [149]:
qml.BasisState(hf_state, wires=range(len(hamiltonian.wires)))

BasisState(array([1, 1, 0, 0, 0, 0]), wires=[0, 1, 2, 3, 4, 5])

In [150]:
g_trial_stat={bitstr: 1}
e_trial_stat_szp1={'101000': 1}
e_trial_stat_szm1={'010100': 1}
e_trial_stat_sz0={'101000': 1,'010100': 1}

In [163]:
electrons = 2
num_qubits = len(hamiltonian.wires)

electrons = 2
S2 = qml.qchem.spin2(electrons, num_qubits)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

sz=qml.qchem.spinz(num_qubits)

dev = qml.device("default.qubit", wires=num_qubits)
print("The original vqe use ",len(doubles)*13 + len(singles)*2," cnot gates")
print("The original vqe use ",len(doubles)*34 + len(singles)*10," gates")

bitstr=''.join(map(str, hf_state))

g_trial_stat={bitstr: 1}
e_trial_stat_szp1={'101000': 1}
e_trial_stat_szm1={'010100': 1}
e_trial_stat_sz0={'101000': 1,'010100': 1}

def bit_to_vec(bitstring_dict,num_qubits):
    vec=np.zeros(2**num_qubits)
    amp=0
    for i in range(2**num_qubits):
        bitstr=bin(i)[2:]
        if len(bitstr) < num_qubits:
            bitstr=(num_qubits-len(bitstr))*'0'+bitstr
        #print(bitstr)
        if bitstr in bitstring_dict:
            vec[i]=bitstring_dict[bitstr]
            amp+=bitstring_dict[bitstr]

    vec=vec/np.sqrt(amp)
    return vec

g_stat=bit_to_vec(g_trial_stat,num_qubits)
e_stat_szp1=bit_to_vec(e_trial_stat_szp1,num_qubits)
e_stat_szm1=bit_to_vec(e_trial_stat_szm1,num_qubits)
e_stat_sz0=bit_to_vec(e_trial_stat_sz0,num_qubits)

def circuit(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    return qml.expval(hamiltonian)

def circuit_sz(state,weights):
    qml.StatePrep(state, wires=range(num_qubits))
    for i in range(len(singles)):
        qml.SingleExcitation(weights[i+len(doubles)], wires=singles[i])
    for i in range(len(doubles)):
        qml.DoubleExcitation(weights[i], wires=doubles[i])

    return qml.expval(sz)


cost_h_fn = qml.QNode(circuit, dev)
Sz_fn = qml.QNode(circuit_sz, dev)

def cost_fn(params):
    return cost_h_fn(g_stat,params)+0.7*cost_h_fn(e_stat_szp1,params)+0.7*cost_h_fn(e_stat_szm1,params)\
            +0.7*cost_h_fn(e_stat_sz0,params)

weights =np.zeros(len(doubles + singles), requires_grad=True)

opt = qml.AdamOptimizer(stepsize=0.5)

i=0
iter=[]
cost_val=[]
Lowest_E=0
best_weights=0

cov=1e-5
E0_pev=0
E1_pev=0
E2_pev=0
E3_pev=0

for _ in range(100):
    weights = opt.step(cost_fn, weights)
    iter.append(i)

    E0=cost_h_fn(g_stat,weights)
    E1=cost_h_fn(e_stat_szp1,weights)
    E2=cost_h_fn(e_stat_sz0,weights)
    E3=cost_h_fn(e_stat_szm1,weights)

    if Lowest_E>E0:
        Lowest_E=E0
        best_weights=weights
    cost_val.append(E0)

    print('iter:',i,' E0:',E0,', E1:',E1,' Sz:',Sz_fn(e_stat_szp1,weights))
    print('E2:',E2,Sz_fn(e_stat_sz0,weights))
    print('E3:',E3,Sz_fn(e_stat_szm1,weights))
    i=i+1

    if np.abs(E0-E0_pev)<cov and np.abs(E1-E1_pev)<cov and np.abs(E2-E2_pev)<cov and np.abs(E3-E3_pev)<cov:
        break
    E0_pev=E0
    E1_pev=E1
    E2_pev=E2
    E3_pev=E3

The original vqe use  60  cnot gates
The original vqe use  176  gates
iter: 0  E0: -1.1245136244417144 , E1: -0.5197267013419172  Sz: 1.0
E2: -0.5197267013422009 -2.220446049250313e-16
E3: -0.5197267013424848 -1.0
iter: 1  E0: -1.0639297123729843 , E1: -0.49027044227216476  Sz: 1.0
E2: -0.4902703187523336 -2.7755575615628914e-17
E3: -0.49027019523250215 -1.0
iter: 2  E0: -1.138553438866692 , E1: -0.5191026900332001  Sz: 1.0
E2: -0.5191025403050402 1.1102230246251565e-16
E3: -0.5191023905768806 -0.9999999999999999
iter: 3  E0: -1.1246874331782997 , E1: -0.5077812736279467  Sz: 1.0
E2: -0.5077480476759046 0.0
E3: -0.507714821723863 -0.9999999999999999
iter: 4  E0: -1.0663507615535388 , E1: -0.4945946659135124  Sz: 1.0000000000000002
E2: -0.4953142646492587 2.7755575615628914e-17
E3: -0.49603386338500544 -0.9999999999999999
iter: 5  E0: -1.115187509394868 , E1: -0.5038215794462672  Sz: 1.0
E2: -0.504922053652689 1.1102230246251565e-16
E3: -0.5060225278591112 -1.0
iter: 6  E0: -1.209949740

In [161]:

electrons = 2
num_qubits = len(hamiltonian.wires)

singles, doubles = qml.qchem.excitations(electrons, num_qubits)
pool_operators=singles+doubles

num_qubits = len(hamiltonian.wires)
hf_state = hf(electrons, num_qubits)

def circuit_1(state,params,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(hamiltonian)

def circuit_sz(state,weights,excitations):
    qml.StatePrep(state, wires=range(num_qubits))
    for i, excitation in enumerate(excitations):
        if len(excitation) == 4:
            qml.DoubleExcitation(params[i], wires=excitation)
        else:
            qml.SingleExcitation(params[i], wires=excitation)
    return qml.expval(sz)

dev = qml.device("default.qubit", wires=num_qubits)#

cost_h_fn = qml.QNode(circuit_1, dev, expansion_strategy="device")

def cost_fn(params, excitations):
    return cost_h_fn(g_stat,params, excitations)+0.7*cost_h_fn(e_stat_szp1,params, excitations)\
            +0.7*cost_h_fn(e_stat_szm1,params, excitations)+0.7*cost_h_fn(e_stat_sz0,params, excitations)


epochs=50

energy=[]

operator_circuits=[]# doubles_select+singles_select
params = [] #[0.0] * len(operator_circuits)
params=np.array(params )

opt = qml.GradientDescentOptimizer(stepsize=0.3)

circuit_gradient = qml.grad(cost_fn, argnum=0)


E0 = cost_h_fn(g_stat,params, operator_circuits)
E1 = cost_h_fn(e_stat_szp1,params, operator_circuits)
E2 = cost_h_fn(e_stat_sz0,params, operator_circuits)
E3 = cost_h_fn(e_stat_szm1,params, operator_circuits)

print(f"Epoch = 0, G Energy = {E0:.8f} Ha, E1 Energy = {E1:.8f} Ha,")
print(f" E2 Energy = {E1:.8f} Ha, E3 Energy = {E2:.8f} Ha,")
print("Number of gates = {}\n".format(len(operator_circuits)))

configs=qml.specs(cost_h_fn)(g_stat,params, excitations=operator_circuits)

#print(configs)
print("Circuit depth {}\n".format(configs['resources'].depth))

conv=1e-5
E0_pev=0
E1_pev=0
E2_pev=0
E3_pev=0
for n in range(epochs):

    Test_set=operator_circuits+pool_operators
    Test_params=list(params)+[0.0]*len(pool_operators)
    Test_params=np.array(Test_params)

    test_grads = circuit_gradient(Test_params,Test_set)
    test_grads = test_grads[len(params):]
    maxpos = np.argmax(np.abs(test_grads))
    #print(np.abs(test_grads[maxpos]))
    max_op=pool_operators[maxpos]
    operator_circuits.append(max_op)
    params=np.append(params, 0.0)

    params,_ = opt.step(cost_fn, params, operator_circuits) # Step 11.
    E0 = cost_h_fn(g_stat,params, operator_circuits)
    E1 = cost_h_fn(e_stat_szp1,params, operator_circuits)
    E2 = cost_h_fn(e_stat_sz0,params, operator_circuits)
    E3 = cost_h_fn(e_stat_szm1,params, operator_circuits)

    print(f"Epoch = {n+1}, G Energy = {E0:.8f} Ha, E1 Energy = {E1:.8f} Ha,")
    print(f" E2 Energy = {E1:.8f} Ha, E3 Energy = {E2:.8f} Ha,")
    print("Number of gates = {}\n".format(len(operator_circuits)))
    configs=qml.specs(cost_h_fn)(g_stat,params, excitations=operator_circuits)
    print("Circuit depth {}\n".format(configs['resources'].depth))

    if np.abs(E0-E0_pev)<cov and np.abs(E1-E1_pev)<cov and np.abs(E2-E2_pev)<cov and np.abs(E3-E3_pev)<cov:
        break
    E0_pev=E0
    E1_pev=E1
    E2_pev=E2
    E3_pev=E3


Epoch = 0, G Energy = -1.21019312 Ha, E1 Energy = -0.51972670 Ha,
 E2 Energy = -0.51972670 Ha, E3 Energy = -0.51972670 Ha,
Number of gates = 0

Circuit depth 1

Epoch = 1, G Energy = -1.21559470 Ha, E1 Energy = -0.51972670 Ha,
 E2 Energy = -0.51972670 Ha, E3 Energy = -0.51972670 Ha,
Number of gates = 1

Circuit depth 2

Epoch = 2, G Energy = -1.22373900 Ha, E1 Energy = -0.51972670 Ha,
 E2 Energy = -0.51972670 Ha, E3 Energy = -0.51972670 Ha,
Number of gates = 2

Circuit depth 3

Epoch = 3, G Energy = -1.22988142 Ha, E1 Energy = -0.51972670 Ha,
 E2 Energy = -0.51972670 Ha, E3 Energy = -0.51972670 Ha,
Number of gates = 3

Circuit depth 4

Epoch = 4, G Energy = -1.23205783 Ha, E1 Energy = -0.51972670 Ha,
 E2 Energy = -0.51972670 Ha, E3 Energy = -0.51972670 Ha,
Number of gates = 4

Circuit depth 5

Epoch = 5, G Energy = -1.23252582 Ha, E1 Energy = -0.51972670 Ha,
 E2 Energy = -0.51972670 Ha, E3 Energy = -0.51972670 Ha,
Number of gates = 5

Circuit depth 6

Epoch = 6, G Energy = -1.23257314 